# Calculating the Channel Capacity Using Convex Optimization

This code (MATLAB) is provided as supplementary material of the lecture Channel Coding - Graph-based Codes.

This code illustrates
* Calculating the channel capacity using convex optimization
* ATTENTION: This code requires a MATLAB-Kernel to run!

Helper function that checks if a channel is weakly symmetric. Input is the channel transition matrix $\boldsymbol{P}$

In [1]:
%%file temp_files/is_weakly_symmetric.m
% check if channel is weakly symmetric
function retval = is_weakly_symmetric(P)
    V = size(P,2);
    W = size(P,1);

    retval = true;
    % first check if all columns are permutations of each other
    col1 = sort(P(:,1));
    for k=2:size(P,2)
        if any(col1 ~= sort(P(:,k)))
            retval = false;
        end
    end
    
    % now check if row sums are equal
    row_sums = sum(P,2);
    if ~all(row_sums == row_sums(1))
        retval = false;
    end    
    
end

File temp_files/is_weakly_symmetric.m created successfully.

Helper function that calculates the channel capacity. First check if the channel is weakly symmetric. If this is not the case, use convex optimization to compute the channel capacity. We use the `fmincon` function. The syntax of this function is:

`x = fmincon(f, x0, A, b, Aeq, beq, l, u)`

and it solves the following optimization problem
$$
\begin{aligned}
\min_{\boldsymbol{x} \in \mathbb{R}^n} \quad & f(\boldsymbol{x}) \\
\text{subject to} \quad 
& \boldsymbol{A}\boldsymbol{x} \leq \boldsymbol{b} \\
& \boldsymbol{A}_{\mathrm{eq}}\boldsymbol{x} = \boldsymbol{b}_{\mathrm{eq}} \\
& \boldsymbol{l}_i \leq x_i \leq \boldsymbol{u}_i,\ i = 1,\ldots, n
\end{aligned}
$$
and $\boldsymbol{x}_0$ is the initial starting point for the optimization.

With our optimization problem
\begin{align*}
\text{maximize}\ \  & (\boldsymbol{1}_{1\times W}\cdot\tilde{\boldsymbol{P}})\boldsymbol{p}_x +\frac{1}{\ln(2)}\boldsymbol{1}_{1\times W}\cdot f_{\textrm{entr.}}(\boldsymbol{P}\boldsymbol{p}_x)\\
\text{subject to}\ &\sum_ip_{x,i} = 1\  \text{and}\  p_{x,i} \geq 0
\end{align*}
we have
\begin{align*}
\boldsymbol{A}_{\mathrm{eq}} &= \begin{pmatrix}1 & 1 & \cdots & 1\end{pmatrix} \\
\boldsymbol{b}_{\mathrm{eq}} &= \begin{pmatrix}1\end{pmatrix} \\
\boldsymbol{l} &= \begin{pmatrix}0 & 0 & \cdots & 0\end{pmatrix} \\
\boldsymbol{u} &= \begin{pmatrix}1 & 1 & \cdots & 1\end{pmatrix} \\
f(\boldsymbol{x}) &= -(\boldsymbol{1}_{1\times W}\cdot\tilde{\boldsymbol{P}})\boldsymbol{p}_x -\frac{1}{\ln(2)}\boldsymbol{1}_{1\times W}\cdot f_{\textrm{entr.}}
\end{align*}
and $\boldsymbol{A}$ and $\boldsymbol{b}$ are not needed/used.

In [2]:
%%file temp_files/channel_capacity.m
% calculate the capacity and the capacity-achieving input distribution for
% a DMC specified by a transition matrix P
function [C, px] = channel_capacity(P)    
    V = size(P,2);
    W = size(P,1);
    
    if is_weakly_symmetric(P)
    % use the formula for weakly symmetric channels
        px = ones(1,V)/V;
        col1 = P(:,1);
        Hc = -sum(col1 .* log2(col1));
        C = log2(W) - Hc;
    else
        P_tilde = P .* log2(P);
        P_tilde(isnan(P_tilde)) = 0;  % 0 * log2(0) = 0
                
        
         % cvx is not available, use built in fmincon function from MATLAB. The interface is a little bit less beautiful than the cvx interface
         myentr = @(x)(max([-x.*log(x)],0));
         [px,C] = fmincon(@(px)(-sum(P_tilde, 1)*px  - sum(myentr(P*px))/log(2)), ...
                          ones(V,1)/V, [], [], ones(1,V), 1, zeros(V,1), ones(V,1), [], ...
                          optimoptions(@fmincon, 'Display', 'off'));
         C = -C;        
    end    
end


File temp_files/channel_capacity.m created successfully.

In [ ]:
addpath('./temp_files');

Starting MATLAB ...


In [4]:
% first example, weakly symmetric channel
fprintf('Weakly symmetric channel:\n');
P = [1/3, 1/3;
     1/2, 1/6;
     1/6, 1/2];

Weakly symmetric channel:


In [5]:
[C, px] = channel_capacity(P);
fprintf('Capacity achieving input distribution:\n');
px
fprintf('Capacity:\n');
C

Unrecognized function or variable 'channel_capacity'.

In [6]:
% second example, non-symmetric channel from lecture
fprintf('Arbitrary channel:\n');
P = [1/2, 1/8;
     1/3, 5/8;
     1/6, 1/4];
 
[C, px] = channel_capacity(P);
fprintf('Capacity achieving input distribution:\n');
px
fprintf('Capacity:\n');
C

Arbitrary channel:


Unrecognized function or variable 'channel_capacity'.

In [20]:
% Example: Z-channel
q = 0.1;
P = [1, q;
     0, 1-q];
[C, px] = channel_capacity(P);
fprintf('Capacity achieving input distribution:\n');
px
h = @(x)(-x.*log2(x) - (1-x).*log2(1-x));
u = ((1-q)+(1-q)*2^(h(q)/(1-q)))^(-1);
fprintf('From analytical formula: [%1.4f %1.4f]\n', 1-u, u);
fprintf('Capacity:\n');
C
fprintf('From analytical formula: %1.4f\n',h(u*(1-q))-u*h(q));

Unrecognized function or variable 'channel_capacity'.

In [21]:
print(pwd)

Error using validateHandleToPrint
No figure to print.

Error in validate (line 17)
pj = validateHandleToPrint(pj); 

Error in print (line 67)
pj = validate( pj );

In [9]:
pwd

ans = '/home/jovyan/work'